# Transpose Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ y := U^T x + y $ where $ U $ is upper triangular.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

## The routine <br> <code> Tmvmult_ut_unb_var1( U, x, y ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U^T x + y $.  The "_ut_" in the name of the routine indicates this is the "upper, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Tmvmult_ut_unb_var1(U, x, y):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.dots(u01, x0, psi1)
        laff.dots(upsilon11, chi1, psi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

# Notice that U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

U before =
[[0.34998846 0.4574239  0.6039462  0.47490646]
 [0.44222852 0.86870014 0.98050337 0.79468858]
 [0.17967689 0.36343742 0.44167373 0.66439613]
 [0.91222072 0.16743694 0.87006324 0.73621101]]
x before =
[[0.66211087]
 [0.50721718]
 [0.25405515]
 [0.48105801]]
y before =
[[0.76282952]
 [0.52704812]
 [0.13683333]
 [0.77923388]]


In [3]:
import numpy as np
laff.copy( y, yold )   # save the original vector y

Tmvmult_ut_unb_var1( U, x, y )

print( 'y after =' )
print( y )

#np.triu makes the matrix upper triangular
print( 'y - ( np.transpose( np.triu( U ) ) * x + yold ) = ' )
print( y - ( np.transpose( np.triu( U ) ) * x + yold ) )

y after =
[[0.99456068]
 [1.2705331 ]
 [1.14625031]
 [2.01970778]]
y - ( np.transpose( np.triu( U ) ) * x + yold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The routine <br> <code> Tmvmult_ut_unb_var2( U, x, y ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U^T x + y $.  The "_ut_" in the name of the routine indicates this is the "upper triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Tmvmult_ut_unb_var2(U, x, y):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, u12t, y2)
        laff.axpy(chi1, upsilon11, psi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

# U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

U before =
[[0.6272008  0.24003664 0.53416193 0.98613958]
 [0.20594849 0.52711139 0.90190458 0.39477776]
 [0.12811288 0.98108702 0.66052014 0.09837707]
 [0.59280778 0.93210418 0.62289281 0.28120628]]
x before =
[[0.97098538]
 [0.32691702]
 [0.4428404 ]
 [0.95229496]]
y before =
[[0.9188828 ]
 [0.15233686]
 [0.63785018]
 [0.0498187 ]]


In [6]:
laff.copy( y, yold )   # save the original vector y

Tmvmult_ut_unb_var2( U, x, y )

print( 'y after =' )
print( y )

#np.triu makes the matrix upper triangular
print( 'y - ( np.transpose( np.triu( U ) ) * x + yold ) = ' )
print( y - ( np.transpose( np.triu( U ) ) * x + yold ) )

y after =
[[1.5278856 ]
 [0.55773061]
 [1.74386657]
 [1.44776206]]
y - ( np.transpose( np.triu( U ) ) * x + yold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.